![UCI](http://mlr.cs.umass.edu/ml/assets/logo.gif)

# Loading in the Data
In this example, we are going to use crossed columns and embedding columns inside of a tensorflow object created with the contrib "learn" library.

However, we will start the process by loading up a dataset with a mix of categorical data and numeric data. This dataset is quite old and has been used many times in machine learning examples: the census data from 1990's. We will use it to predict if a person will earn over or under 50k per year.

- https://archive.ics.uci.edu/ml/datasets/Census-Income+(KDD)

In [1]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline 
%load_ext memory_profiler
from sklearn.metrics import make_scorer
from scipy.special import expit
import time
import math
import random
from memory_profiler import memory_usage
from sklearn import metrics as mt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from scipy import interp

target_classifier = 'Spending on gadgets'
df = pd.read_csv('responses.csv', sep=",")
desired_features = ["Music","Dance","Folk","Country","Classical music","Musical",
    "Pop","Rock","Metal or Hardrock","Punk","Hiphop, Rap","Reggae, Ska",
    "Swing, Jazz","Rock n roll","Alternative","Latino","Techno, Trance",
    "Opera","Movies","Horror","Thriller","Comedy","Romantic","Sci-fi","War",
    "Fantasy/Fairy tales","Animated","Documentary","Western","Action","History",
    "Psychology","Politics","Mathematics","Physics","Internet","PC","Economy Management",
    "Biology","Chemistry","Reading","Geography","Foreign languages","Medicine","Law",
    "Cars","Art exhibitions","Religion","Countryside, outdoors","Dancing",
    "Musical instruments","Writing","Passive sport","Active sport","Gardening","Celebrities",
    "Shopping","Science and technology","Theatre","Fun with friends","Adrenaline sports",
    "Pets","Smoking","Alcohol","Healthy eating","Spending on gadgets","Age","Height","Weight",
    "Number of siblings","Gender","Left - right handed","Education","Village - town","House - block of flats"
]

df = df[desired_features]
df

,Music,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,Punk,...,Spending on gadgets,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Village - town,House - block of flats
0,5.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,1.0,...,1,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,village,block of flats
1,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,4.0,...,5,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,city,block of flats
2,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,4.0,...,4,20.0,176.0,67.0,2.0,female,right handed,secondary school,city,block of flats
3,5.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,4.0,...,4,22.0,172.0,59.0,1.0,female,right handed,college/bachelor degree,city,house/bungalow
4,5.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,2.0,...,2,20.0,170.0,59.0,1.0,female,right handed,secondary school,village,house/bungalow
5,5.0,2.0,3.0,2.0,3.0,3.0,2.0,5.0,5.0,3.0,...,4,20.0,186.0,77.0,1.0,male,right handed,secondary school,city,block of flats
6,5.0,5.0,3.0,1.0,2.0,2.0,5.0,3.0,1.0,1.0,...,1,20.0,177.0,50.0,1.0,female,right handed,secondary school,village,house/bungalow
7,5.0,3.0,2.0,1.0,2.0,2.0,4.0,5.0,1.0,2.0,...,3,19.0,184.0,90.0,1.0,male,right handed,college/bachelor degree,city,house/bungalow
8,5.0,3.0,1.0,1.0,2.0,4.0,3.0,5.0,5.0,1.0,...,3,18.0,166.0,55.0,1.0,female,right handed,secondary school,city,house/bungalow
9,5.0,2.0,5.0,2.0,2.0,5.0,3.0,5.0,2.0,3.0,...,2,19.0,174.0,60.0,3.0,female,right handed,secondary school,city,block of flats


In [2]:
print(df['Spending on gadgets'])

0       1
1       5
2       4
3       4
4       2
5       4
6       1
7       3
8       3
9       2
10      2
11      1
12      2
13      3
14      1
15      5
16      3
17      2
18      5
19      3
20      4
21      2
22      4
23      2
24      5
25      1
26      4
27      2
28      1
29      2
       ..
980     4
981     2
982     3
983     3
984     4
985     3
986     1
987     4
988     5
989     1
990     4
991     3
992     1
993     4
994     3
995     2
996     3
997     2
998     2
999     3
1000    4
1001    2
1002    3
1003    4
1004    4
1005    3
1006    5
1007    2
1008    3
1009    1
Name: Spending on gadgets, dtype: int64


In [3]:
df.rename(columns={'Reggae, Ska': 'ReggaeSka', 'Swing, Jazz': 'SwingJazz', 'Classical music': 'Classicalmusic', 'Metal or Hardrock': 'MetalorHardrock', 'Hiphop, Rap': 'HiphopRap', 'Rock n roll': 'Rocknroll', 'Techno, Trance': 'TechnoTrance', 'Sci-fi': 'Scifi', 'Fantasy/Fairy tales': 'Fantasy', 'Economy Management': 'EconomyManagement', 'Foreign languages': 'Foreignlanguages', 'Art exhibitions': 'Artexhibitions', 'Countryside, outdoors': 'outdoors', 'Musical instruments': 'Musicalinstruments', 'Passive sport': 'Passivesport', 'Active sport': 'Activesport', 'Science and technology': 'Scienceandtechnology', 'Fun with friends': 'Funwithfriends', 'Adrenaline sports': 'Adrenalinesports', 'Healthy eating': 'Healthyeating', 'Spending on gadgets': 'Spendingongadgets', 'Number of siblings': 'Numberofsiblings', 'Left - right handed': 'Leftrighthanded', 'Village - town': 'Villagetown', 'House - block of flats': 'Houseblockofflats'}, inplace=True)

In [4]:
from copy import deepcopy
df_train = deepcopy(df)
df_test = deepcopy(df)

In [5]:

import numpy as np

# let's just get rid of rows with any missing data
# and then reset the indices of the dataframe so it corresponds to row number
df_train.replace(to_replace=' ?',value=np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.reset_index()

df_test.replace(to_replace=' ?',value=np.nan, inplace=True)
df_test.dropna(inplace=True)
df_test.reset_index()

df_test.head()

,Music,Dance,Folk,Country,Classicalmusic,Musical,Pop,Rock,MetalorHardrock,Punk,...,Spendingongadgets,Age,Height,Weight,Numberofsiblings,Gender,Leftrighthanded,Education,Villagetown,Houseblockofflats
0,5.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,1.0,...,1,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,village,block of flats
1,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,4.0,...,5,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,city,block of flats
2,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,4.0,...,4,20.0,176.0,67.0,2.0,female,right handed,secondary school,city,block of flats
3,5.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,4.0,...,4,22.0,172.0,59.0,1.0,female,right handed,college/bachelor degree,city,house/bungalow
4,5.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,2.0,...,2,20.0,170.0,59.0,1.0,female,right handed,secondary school,village,house/bungalow


## Processing
For preprocessing, we are going to fix a few issues in the dataset. 

- This first includes the use of "50K." instead of "50K" in the test set. 
- Next, we will encode the categorical features as integers (later on we will encode one hot)
- Finally, we will make certain all the continuous data is scaled properly

In [6]:
categorical_headers = list(df.select_dtypes(include=['object']).columns)
numeric_headers = list(df.select_dtypes(include=['float']).columns) + list(df.select_dtypes(include=['int']).columns)


In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

encoders = dict() 

for col in categorical_headers:
    df_train[col] = df_train[col].str.strip()
    df_test[col] = df_test[col].str.strip()
    
    if col=='Spendingongadgets':
        tmp = LabelEncoder()
        df_train[col] = tmp.fit_transform(df_train[col])
        df_test[col] = tmp.transform(df_test[col])
    else:
        encoders[col] = LabelEncoder()
        df_train[col+'_int'] = encoders[col].fit_transform(df_train[col])
        df_test[col+'_int'] = encoders[col].transform(df_test[col])

print(numeric_headers)
for col in numeric_headers:
    df_train[col] = df_train[col].astype(np.float)
    df_test[col] = df_test[col].astype(np.float)
    
    ss = StandardScaler()
    df_train[col] = ss.fit_transform(df_train[col].values.reshape(-1, 1))
    df_test[col] = ss.transform(df_test[col].values.reshape(-1, 1))

['Music', 'Dance', 'Folk', 'Country', 'Classicalmusic', 'Musical', 'Pop', 'Rock', 'MetalorHardrock', 'Punk', 'HiphopRap', 'ReggaeSka', 'SwingJazz', 'Rocknroll', 'Alternative', 'Latino', 'TechnoTrance', 'Opera', 'Movies', 'Horror', 'Thriller', 'Comedy', 'Romantic', 'Scifi', 'War', 'Fantasy', 'Animated', 'Documentary', 'Western', 'Action', 'History', 'Psychology', 'Politics', 'Mathematics', 'Physics', 'Internet', 'PC', 'EconomyManagement', 'Biology', 'Chemistry', 'Reading', 'Geography', 'Foreignlanguages', 'Medicine', 'Law', 'Cars', 'Artexhibitions', 'Religion', 'outdoors', 'Dancing', 'Musicalinstruments', 'Writing', 'Passivesport', 'Activesport', 'Gardening', 'Celebrities', 'Shopping', 'Scienceandtechnology', 'Theatre', 'Funwithfriends', 'Adrenalinesports', 'Pets', 'Healthyeating', 'Age', 'Height', 'Weight', 'Numberofsiblings', 'Spendingongadgets']


In [8]:
print(df_train['Spendingongadgets'])

0      -1.435513
1       1.640729
2       0.871668
3       0.871668
4      -0.666453
5       0.871668
6      -1.435513
7       0.102608
9      -0.666453
10     -0.666453
11     -1.435513
12     -0.666453
13      0.102608
14     -1.435513
15      1.640729
16      0.102608
17     -0.666453
18      1.640729
19      0.102608
20      0.871668
21     -0.666453
23     -0.666453
24      1.640729
25     -1.435513
26      0.871668
28     -1.435513
29     -0.666453
30      0.871668
31     -0.666453
32     -1.435513
          ...   
971    -0.666453
972     0.871668
973     0.871668
974    -1.435513
976     0.102608
977    -0.666453
978     0.102608
979     1.640729
981    -0.666453
982     0.102608
983     0.102608
984     0.871668
985     0.102608
986    -1.435513
989    -1.435513
990     0.871668
991     0.102608
992    -1.435513
994     0.102608
995    -0.666453
998    -0.666453
1000    0.871668
1001   -0.666453
1002    0.102608
1003    0.871668
1004    0.871668
1005    0.102608
1007   -0.6664

___

![tf](https://wiki.tum.de/download/attachments/25009442/tensor-flow_opengraph_h.png?version=1&modificationDate=1485888308193&api=v2)

# Starting Tensorflow
Now that we have processed the data, let's grab numpy matrices of the features we would like to predict with. In particular, we will convert everything to 32 bits (as a lot of Tensorflow is written with this arch in mind). 

In [9]:
# let's start as simply as possible, without any feature preprocessing
categorical_headers_ints = [x+'_int' for x in categorical_headers]

# we will forego one-hot encoding right now and instead just scale all inputs
feature_columns = categorical_headers_ints+numeric_headers
#X_train =  ss.fit_transform(df_train[feature_columns].values).astype(np.float32)
X =  ss.transform(df_test[feature_columns].values).astype(np.float32)
y = df_train['Spendingongadgets'].values.astype(np.int)


print(feature_columns)
# split into more X_train X_test y_train y_test here

['Smoking_int', 'Alcohol_int', 'Gender_int', 'Leftrighthanded_int', 'Education_int', 'Villagetown_int', 'Houseblockofflats_int', 'Music', 'Dance', 'Folk', 'Country', 'Classicalmusic', 'Musical', 'Pop', 'Rock', 'MetalorHardrock', 'Punk', 'HiphopRap', 'ReggaeSka', 'SwingJazz', 'Rocknroll', 'Alternative', 'Latino', 'TechnoTrance', 'Opera', 'Movies', 'Horror', 'Thriller', 'Comedy', 'Romantic', 'Scifi', 'War', 'Fantasy', 'Animated', 'Documentary', 'Western', 'Action', 'History', 'Psychology', 'Politics', 'Mathematics', 'Physics', 'Internet', 'PC', 'EconomyManagement', 'Biology', 'Chemistry', 'Reading', 'Geography', 'Foreignlanguages', 'Medicine', 'Law', 'Cars', 'Artexhibitions', 'Religion', 'outdoors', 'Dancing', 'Musicalinstruments', 'Writing', 'Passivesport', 'Activesport', 'Gardening', 'Celebrities', 'Shopping', 'Scienceandtechnology', 'Theatre', 'Funwithfriends', 'Adrenalinesports', 'Pets', 'Healthyeating', 'Age', 'Height', 'Weight', 'Numberofsiblings', 'Spendingongadgets']


In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

num_cv_iterations = 3
num_instances = len(y)
cv_object = StratifiedShuffleSplit(n_splits=num_cv_iterations,test_size = 0.2)

print(cv_object)

StratifiedShuffleSplit(n_splits=3, random_state=None, test_size=0.2,
            train_size=None)


In [11]:
for train_indices, test_indices in cv_object.split(X,y): 

            X_train = (X[train_indices])
            y_train = y[train_indices]

            X_test = (X[test_indices])
            y_test = y[test_indices]

Now let's import the tensorflow contrib libraries. These libraries are the workhorses for tensorflow's simplified interface. Please note that this was previously called "SKFlow" and might one day be replaced by the simplified syntax of Keras. Keras will received native TF support and be included in tf.contrib but as of the creation of this notebook, that is not the case. As such, we will be using syntax for the contrib library as of Rev 1.0 of tensorflow. 

- Please also note that keras is getting added to the core tensorflow contribution library such that this command will work:
 - `from tensorflow.contrib import keras`
- As such, if you want to use the Keras API, it will soon be supported! Even so, its unclear what syntax changes and additions will be made to the existing Keras tools, so we will forge ahead using the "learn" API

When using the learn API, we will still be using many of the native tensorflow functions, so we 
- `import tensorflow as tf` to get access to the entire API when needed
- `from tensorflow.contrib import learn` to get access to many of the wrappers and simplifications for using tensorflow. This library really helps with creating models and using some out-of-the-box networks (like a shallow/deep MLP with fully connected layers)
- `from tensorflow.contrib import layers` will be used to get access to some common neural network layer types and activations 
- `from tensorflow.contrib.learn.python import SKCompat` this will be used to give us a very familir interface as used in sklearn.

In [12]:
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers
from tensorflow.contrib.learn.python import SKCompat
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
tf.logging.set_verbosity(tf.logging.WARN) # control the verbosity of tensor flow

![tflearn](http://www.kdnuggets.com/wp-content/uploads/skflow.jpg)

## An example similar to Sklearn
We will start with creating a model that is similar to what we have seen in scikit-learn. 

In [13]:
%%time
# we need to tell tensorflow how many inputs to expect and what the data types will be
# for this early example, everything is just numeric, real valued
features_tf = [layers.real_valued_column('', dimension=X_train.shape[1])]
clf = SKCompat(# wrap with SKCompat for easy usage like sklearn
            learn.DNNClassifier(hidden_units=[50], feature_columns=features_tf)
        )

clf.fit(X_train,y_train,steps=100)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
CPU times: user 5.76 s, sys: 101 ms, total: 5.86 s
Wall time: 5.85 s


In [14]:
from sklearn import metrics as mt

yhat = clf.predict(X_test)
# notice that the output needs some interpretation
# as its not completely the same as sklearn
yhat = yhat['classes']
print(mt.confusion_matrix(y_test,yhat),
      mt.accuracy_score(y_test,yhat))

[[  0  27   0]
 [  0 106   0]
 [  0  22   0]] 0.683870967742


In [15]:
%%time
# we can also custimize the classifier somewhat:
clf = SKCompat(# wrap with SKCompat for easy usage like sklearn
            learn.DNNClassifier(hidden_units=[50], 
                                feature_columns=features_tf,
                                activation_fn=tf.nn.sigmoid 
                                # tf.tanh, tf.sigmoid, tf.nn.relu, tf.nn.softmax etc.
                                )
        )

clf.fit(X_train,y_train,steps=1000)

yhat = clf.predict(X_test)['classes']
print(mt.confusion_matrix(y_test,yhat),
      mt.accuracy_score(y_test,yhat))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
[[  0  27   0]
 [  0 106   0]
 [  0  10  12]] 0.761290322581
CPU times: user 7.68 s, sys: 406 ms, total: 8.09 s
Wall time: 7.24 s


___
## Adding more customization
This type of architecture is fine to use, but suffers from many limitations: if its not implemented by the `learn` API, then we can't use the architecture or customize it easily. To get around this, we, instead, will use the `Estimator` class. To use this class, we need to define a "model function" that creates the neural network architecture. The output of this model needs to be very specific: a three tuple of 
- (1) predictions in a format that the end user will need to use
- (2) loss function implemented as tf graph 
- (3)  the optimization operation to perform on the graph. 

For this particular optimization, let's go beyond SGD and use Adagrad. There are many excellent explanations of different optimizers, for instance:
- http://sebastianruder.com/optimizing-gradient-descent/

There is a lot of code here that might be new, but essentially we are starting to get down to the core functionality of tensorflow (even though still using the learn API).

In [16]:

# let's start by just using the tflearn library out of the box on the data we have
def my_model(features, targets, mode):
    # the prototype for this function is as follows
    # input:  (features, targets) 
    # output: (predictions, loss, train_op)
    
    # =====SETUP ARCHITECTURE=====
    # we can use functions from learn to add layers and complexity to the model
    # pass features through one hidden layer with relu activation
    features = layers.relu(features, num_outputs=50) 
    # now pass the features through a fully connected layer
    features = layers.fully_connected(features, num_outputs=1) 
    # and pass them through a sigmoid activation
    output_layer = tf.sigmoid(features) 
    # reshape the output to be one dimensional
    predictions = tf.reshape(output_layer, [-1])
    
    # depending on the mode, we may not want to evaluate these
    loss_mse = None
    train_op = None
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        # =====LOSS=======
        # we want to use MSE as our loss function, but could also choose 
        # cross entropy, or other objective functions here
        loss_mse = tf.losses.mean_squared_error(targets, predictions) 
    
    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        # =====OPTIMIZER PARAMS========
        # now let's setup how we want thing to optimize 
        train_op = layers.optimize_loss(
            loss=loss_mse, 
            global_step=tf.contrib.framework.get_global_step(),
            optimizer='Adagrad', # adaptive gradient, so that the learning rate is not SO important
            learning_rate=0.1)
    
    # what format to have the output in when calling clf.predict?
    predictions_out = predictions>0.5
    return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions_out, loss=loss_mse, train_op=train_op)


Now that we have defined the model, its as simple as using it in a very familiar syntax:

In [17]:
%%time

from sklearn.metrics import confusion_matrix, accuracy_score

# now let train the estimator
# we can wrap the tf estimator in the SKCompat class
# so that we can use similar syntax to to SKLearn 
clf = SKCompat(learn.Estimator(model_fn=my_model))
clf.fit(X_train, y_train, steps=5000, batch_size=32)

yhat = clf.predict(X_test)
print(confusion_matrix(y_test,yhat), 
      accuracy_score(y_test,yhat))

print(X_test.shape[1], len(yhat))

[[  0  27   0]
 [  0 106   0]
 [  0  22   0]] 0.683870967742
75 155
CPU times: user 10.3 s, sys: 1.7 s, total: 12 s
Wall time: 8.51 s


___

### Note [start review here]

## Handling different feature types

You may have noticed that we were not handling the categorical features correctly. Really we need to one hot encode the categorical features
unfortunately, this requires using part of the API that breaks some conventions of SKLearn. In this example, we will use the integer categorical labels to get one hot encoded examples.

The advantage of this method is that we can write our custom preprocessing steps and use them straight in the pipeline of the classifier. However, the syntax for this is verbose and fairly ugly. So, let's get started!

First, we need a process input function that performs all the tensoflow preparation of the data and returns a dictionary of the name and values of each feature column. It should also return the target column as a tf.constant

In [18]:
# Let's start with the TF example (manipulated to work with new syntax)
# https://www.tensorflow.org/tutorials/wide_and_deep
def process_input(df, label_header, categ_headers, numeric_headers):
    # input: what ever you need it to be
    # output: (dict of feature columns as tensors), (labels as tensors)
    
    # ========Process Inputs=========
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.expand_dims( # make it a column vector
                            tf.cast( # cast to a float32
                                tf.constant(df[k].values), 
                                tf.float32), 
                            1)
                       for k in numeric_headers}
    
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored as constant Tensors (numeric)
    # then use tensor flow to one hot encode them using the given number of classes 
    # name of encoder is **_int need to map only to **
    categorical_cols = {k: tf.one_hot(indices=tf.constant(df[k].values),
                                      depth=len(encoders[k[:-4]].classes_)) 
                        for k in categ_headers}
    
    # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols)
    feature_cols.update(categorical_cols)
    
    # Convert the label column into a constant Tensor.
    label = None
    if label_header is not None:
        label = tf.constant(df[label_header].values)
        
    return feature_cols, label


Now, when we want to use the classifier, we tell the model how it needs to parse the inputs. For each numeric feature, we simply need to convert the tensors into column vectors (see below). For the categrocial vectors, TF has a convenient "one_hot" function that we can use. The remainder of the model selection stays the same. 

In [19]:
# update the model to take input features as a dictionary
def my_model(dict_features, targets, mode):
    # the prototype for this function is as follows
    # input:  (features, targets) 
    # output: (predictions, loss, train_op)
    
    #=======DECODE FEATURES================
    # now let's combine the tensors from the input dictionary
    # into a list of the feature columns
    features = []
    #features = [dict_features[x] for x in numeric_headers+categorical_headers_ints]
    for col in numeric_headers:
        features.append(dict_features[col])
    
    # also add in the one hot encoded features
    for col in categorical_headers_ints:
        features.append(dict_features[col])
    
    # now we can just combine all the features together
    features = tf.concat(values=features,axis=1)
    
    # =====SETUP ARCHITECTURE=====
    # we can use functions from learn to add layers and complexity to the model
    # pass features through one hidden layer with relu activation
    features = layers.relu(features, num_outputs=50) 
    # now pass the features through a fully connected layer
    features = layers.fully_connected(features, num_outputs=1) 
    # and pass them through a sigmoid activation
    output_layer = tf.sigmoid(features) 
    # reshape the output to be one dimensional
    predictions = tf.reshape(output_layer, [-1])
    
    # depending on the mode, we may not want to evaluate these
    loss_mse = None
    train_op = None
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        # =====LOSS=======
        # we want to use MSE as our loss function
        loss_mse = tf.losses.mean_squared_error(targets, predictions) 
    
    if mode == learn.ModeKeys.TRAIN:
        # =====OPTIMIZER PARAMS========
        # now let's setup how we want thing to optimize 
        train_op = layers.optimize_loss(
            loss=loss_mse, 
            global_step=tf.contrib.framework.get_global_step(),
            optimizer='Adagrad', # adaptive gradient, so that the learning rate is not SO important 
            learning_rate=0.1)
    
    # what format to have the output in when calling clf.predict?
    predictions_out = predictions>0.5
    
    return model_fn_lib.ModelFnOps(
      mode=mode, predictions={'Spendingongadgets':predictions_out}, loss=loss_mse, train_op=train_op)

Now to call the estimator with our custom pre-processing step we need to wrap the preprocessing inside another function. This is easy to do with a simple lambda, but you could also use another wrapper function that takes no inputs. 

In [23]:
%%time
clf = learn.Estimator(model_fn=my_model)

# when we provide the process function, they expect us to control the mini-batch
clf.fit(input_fn=
        lambda:process_input(df_train,'Spendingongadgets',categorical_headers_ints, numeric_headers), 
        steps=500)

CPU times: user 6.21 s, sys: 258 ms, total: 6.47 s
Wall time: 5.38 s


In [27]:
yhat = clf.predict(input_fn=
                   lambda:process_input(X_test,None,categorical_headers_ints, numeric_headers))
# the output is now an iterable value, so we need to step over it
yhat = [x['Spendingongadgets'] for x in yhat]
print(confusion_matrix(y_test,yhat),
      accuracy_score(y_test,yhat))

/Users/vantran/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
len(y_train)

In [ ]:
aucs = []
fprs = []
tprs = []

ind_yhat = yhat
mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []
perclass_mean_tpr = 0.0
roc_auc = 0
classes = np.unique(y_train)
    # get the mean fpr and tpr, per class
for j in classes:
    fpr, tpr, thresholds = roc_curve(y_test,
                                     yhat,
                                     pos_label=j)
    perclass_mean_tpr += interp(mean_fpr, fpr, tpr)
    perclass_mean_tpr[0] = 0.0
    roc_auc += auc(fpr, tpr)
    fprs.append(fpr)
    tprs.append(tpr)
    aucs.append(auc(fpr, tpr))
plt.figure()
plt.title("Performance of our standard MLP")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

for i in range(0,3):
    plt.plot(fprs[i],tprs[i],label='Class %d (area = %0.2f)'
               % (i+1, aucs[i]))
    plt.legend(loc='best')

plt.show()

So the confusion matrix is doing pretty well! But we still are just using an MLP with one hidden layer. We really want to take advantage of the crossed columns and embeddings that are possible with tensorflow. That takes yet another syntax change in how we create out tensorflow object. 

### Note: ended wide/deep lecture here. 

## [Back to Slides]

![asdfasfd](https://www.tensorflow.org/images/wide_n_deep.svg)
____
# Adding Crossed Columns
For this example, we are going to combine our `learn` syntax with the syntax we have just gone over. 

In [ ]:
# Now lets create a wide model 
# https://www.tensorflow.org/tutorials/wide_and_deep
def process_input(df, label_header, categ_headers, numeric_headers):
    # input: what ever you need it to be
    # output: (dict of feature columns as tensors), (labels as tensors)
    
    # ========Process Inputs=========
    # not much changes here, except we leave the numerics as tc.constants
    continuous_cols = {k: tf.constant(df[k].values) for k in numeric_headers}
      
    # and we shift these tensors to be sparse one-hot encoded values
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(
                              indices=[[i, 0] for i in range(df[k].size)],
                              values=df[k].values,
                              dense_shape=[df[k].size, 1])
                        for k in categ_headers}
    
    # Merges the two dictionaries into one.
    feature_cols = dict(categorical_cols)
    feature_cols.update(continuous_cols)
    
    # Convert the label column into a constant Tensor.
    label = None
    if label_header is not None:
        label = tf.constant(df[label_header].values)
        
    return feature_cols, label


In [ ]:
# update the model to take input features as a dictionary
def setup_wide_columns():
    # let's create the column structure that the learn API can expect
    
    wide_columns = []
    # add in each of the categorical columns
    for col in categorical_headers:
        wide_columns.append(layers.sparse_column_with_keys(col, keys=encoders[col].classes_))
        
    # also add in some specific crossed columns
    cross_columns = [('Smoking','Alcohol')]
    for tup in cross_columns:
        wide_columns.append(
            layers.crossed_column(
                [layers.sparse_column_with_keys(tup[0], keys=encoders[tup[0]].classes_),
                 layers.sparse_column_with_keys(tup[1], keys=encoders[tup[1]].classes_)],
            hash_bucket_size=int(1e4))
        )
                        
    return wide_columns

In [ ]:
print(categorical_headers)
print(numeric_headers)

In [ ]:
%%time

# ignore all the deprecations that the learn API needs to deal with... ugh
tf.logging.set_verbosity(tf.logging.ERROR)

# setup 
wide_columns = setup_wide_columns()
input_wrapper = lambda:process_input(df_train,'Spendingongadgets',categorical_headers, numeric_headers)
output_wrapper = lambda:process_input(df_test,None,categorical_headers, numeric_headers)

clf = learn.LinearClassifier(feature_columns=wide_columns)

# when we provide the process function, they expect us to control the mini-batch
clf.fit(input_fn=input_wrapper, steps=300)

yhat = clf.predict(input_fn=output_wrapper)
# the output is now an iterable value, so we need to step over it
yhat = [x for x in yhat]
print(confusion_matrix(y_test,yhat),accuracy_score(y_test,yhat))

Wow! That is just using crossed columns and a one layer Linear Classifer! So memorization works fairly well here. 

Now let's try a deeper architecture with dense embeddings for the categorical features, as described in lecture.

___

## Using Dense embeddings in a deeper network

In [ ]:
# update the model to take input features as a dictionary
def setup_deep_columns():
    # now make up the deep columns
    
    deep_columns = []
    # add in each of the categorical columns to both wide and deep features
    for col in categorical_headers:
        
        tmp = layers.sparse_column_with_keys(col, keys=encoders[col].classes_)
        
        deep_columns.append(
            layers.embedding_column(tmp, dimension=8)
        )
        
        
    # and add in the regular dense features 
    for col in numeric_headers:
        deep_columns.append(
            layers.real_valued_column(col)
        )
                    
    return deep_columns

In [ ]:
%%time 

# setup deep columns
deep_columns = setup_deep_columns()
clf = learn.DNNClassifier(feature_columns=deep_columns, hidden_units=[100, 50])

clf.fit(input_fn=input_wrapper, steps=300)

yhat = clf.predict(input_fn=output_wrapper)
# the output is now an iterable value, so we need to step over it
yhat = [x for x in yhat]
print(confusion_matrix(y_test,yhat),
      accuracy_score(y_test,yhat))

So the DNN with feature embeddings is also fairly capable (best thus far). For the final example, lets combine the classifiers together like we talked about in lecture.
___

## Combining Crossed Linear Classifier and Deep Embeddings
Now its just a matter of setting the wide and deep columns for tensorflow. After which, we can use the combined classifier which is already implemented! `learn.DNNLinearCombinedClassifier`

In [ ]:
# update the model to take input features as a dictionary
def setup_wide_deep_columns():
    # the prototype for this function is as follows
    # input:  (features, targets) 
    # output: (predictions, loss, train_op)
    
    wide_columns = []
    deep_columns = []
    # add in each of the categorical columns to both wide and deep features
    for col in categorical_headers:
        wide_columns.append(
            layers.sparse_column_with_keys(col, keys=encoders[col].classes_)
        )
        
        dim = round(np.log2(len(encoders[col].classes_)))
        deep_columns.append(
            layers.embedding_column(wide_columns[-1], dimension=dim)
        )
        
    # also add in some specific crossed columns
    cross_columns = [('Smoking','Alcohol')]
    for tup in cross_columns:
        wide_columns.append(
            layers.crossed_column(
                [layers.sparse_column_with_keys(tup[0], keys=encoders[tup[0]].classes_),
                 layers.sparse_column_with_keys(tup[1], keys=encoders[tup[1]].classes_)],
            hash_bucket_size=int(1e4))
        )
        
        
    # and add in the regular dense features 
    for col in numeric_headers:
        deep_columns.append(
            layers.real_valued_column(col)
        )
                    
    return wide_columns, deep_columns

In [ ]:
%%time

wide_columns, deep_columns = setup_wide_deep_columns()
clf = learn.DNNLinearCombinedClassifier(
                        linear_feature_columns=wide_columns,
                        dnn_feature_columns=deep_columns,
                        dnn_hidden_units=[100, 50])


clf.fit(input_fn=input_wrapper, steps=2500)

yhat = clf.predict(input_fn=output_wrapper)
# the output is now an iterable value, so we need to step over it
yhat = [x for x in yhat]
print(confusion_matrix(y_test,yhat),accuracy_score(y_test,yhat))

In [ ]:
aucs = []
fprs = []
tprs = []

ind_yhat = yhat
mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []
perclass_mean_tpr = 0.0
roc_auc = 0
classes = np.unique(y_train)
    # get the mean fpr and tpr, per class
for j in classes:
    fpr, tpr, thresholds = roc_curve(y_test,
                                     yhat,
                                     pos_label=j)
    perclass_mean_tpr += interp(mean_fpr, fpr, tpr)
    perclass_mean_tpr[0] = 0.0
    roc_auc += auc(fpr, tpr)
    fprs.append(fpr)
    tprs.append(tpr)
    aucs.append(auc(fpr, tpr))
plt.figure()
plt.title("Performance of our best wide and deep network")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

for i in range(0,3):
    plt.plot(fprs[i],tprs[i],label='Class %d (area = %0.2f)'
               % (i+1, aucs[i]))
    plt.legend(loc='best')

plt.show()